### Import Packages

In [ ]:
import nltk; nltk.download('stopwords')
import numpy as np
from sklearn import model_selection
import pandas as pd
import re, nltk, spacy, gensim

# Scikit-Learn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plot tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [23]:
df1 = pd.read_excel('Management_Opportunity_Buzz.xlsx', sheet_name='Relevant')



/anaconda3/lib/python3.7/site-packages/defusedxml/ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
/anaconda3/lib/python3.7/site-packages/defusedxml/ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
/anaconda3/lib/python3.7/site-packages/defusedxml/ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
/anaconda3/lib/python3.7/site-packages/defusedxml/ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
/anaconda3/lib/python3.7/site-packages/defusedxml/ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
/anaconda3/lib/python3.7/site-packages/defusedxml/ElementTre

In [57]:
print(df1.Sentences.unique())
df1.head(15)

[" we reported last quarter that we have a penetration in terms of patients of about 59%. We have seen in the fourth quarter this penetration is stable. And is something that, in some way, was expected because it's a quarter in where most of the use is in acute, while   I'm sorry, chronic, while we have seen increase in the penetration because of the use of acute and seasonal."
 "0 if and when   if it came out as we expect at the end of 2018. And the reality is our share is really sort of on a quarter-over-quarter basis over the last two quarters or three quarters has been relatively stable. And we did say we're down a little bit over 4Q 2016 to 4Q 2017."
 "0%. And so we are still in the emerging economy, and we are still   we have a really conservative approach to our provisioning and actually based on this year's stable results, and our conservative approach is really a big caution for all our shareholders. Second as you really well know one of the peers have recently provided 1."
 .

,Sentences,stable,stability,steady,maintain,predictable,earnings,dividend,regular,environment,...,solid,progress,sentiment_negative,sentiment_neutral,sentiment_positive,sentiment_NA,not relevant,relevant,very relevant,extremely relevant
0,we reported last quarter that we have a penet...,1,0,0,0,0,0,0,0,0,...,0,0,0.000000,0.526316,0.473684,0.000000,0.000000,0.842105,0.157895,0.0
1,0 if and when if it came out as we expect at...,1,0,0,0,0,0,0,0,0,...,0,0,0.086957,0.391304,0.347826,0.173913,0.173913,0.695652,0.130435,0.0
2,0%. And so we are still in the emerging econom...,1,0,0,0,0,0,0,0,0,...,0,0,0.000000,0.692308,0.153846,0.153846,0.153846,0.615385,0.230769,0.0
3,02 per share on earnings. Our objective in 201...,1,0,0,1,0,1,0,0,0,...,0,0,0.117647,0.117647,0.411765,0.352941,0.352941,0.470588,0.176471,0.0
4,04) to our team. From our last earnings releas...,0,0,1,0,0,1,0,0,0,...,0,0,0.000000,0.320000,0.560000,0.120000,0.120000,0.640000,0.240000,0.0
5,05 a share in 2018 as well. And we're now more...,1,0,0,0,1,1,1,0,0,...,0,0,0.000000,0.666667,0.333333,0.000000,0.000000,1.000000,0.000000,0.0
6,05 billion as the capital expenditure for 2017...,0,0,1,0,0,0,0,0,0,...,0,0,0.000000,0.600000,0.250000,0.150000,0.150000,0.700000,0.150000,0.0
7,05 which was confirmed by our supervisory boar...,1,0,0,1,0,0,1,0,0,...,0,0,0.000000,0.631579,0.368421,0.000000,0.000000,0.842105,0.157895,0.0
8,"06) as Larry laid out, the expectations for 20...",1,0,0,0,0,0,0,0,0,...,0,0,0.000000,0.470588,0.529412,0.000000,0.000000,0.823529,0.176471,0.0
9,"08). From slide 15, our performance in the fou...",1,0,0,0,0,0,0,0,0,...,0,0,0.000000,0.500000,0.450000,0.050000,0.050000,0.800000,0.150000,0.0


### Data Preprocessing

In [3]:

data = df1.Sentences.values.tolist()

data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

[' we reported last quarter that we have a penetration in terms of patients of '
 'about 59%. We have seen in the fourth quarter this penetration is stable. '
 'And is something that, in some way, was expected because its a quarter in '
 'where most of the use is in acute, while   Im sorry, chronic, while we have '
 'seen increase in the penetration because of the use of acute and seasonal.']


In [24]:

stopwords = nltk.corpus.stopwords.words('english')
new_stopwords = ['included','including','includes','one','two','three','general','generally','way','better','need','really','believe','provide','say','saying','says','include','said','also','would','could','end','recent','recently','us','like','make','may','look','still','come','see','across','even','much','get','allow','allows','allowed','allowing','well','go','gone','going','that','thats','take','thing','come','move','give','help','put','dont','people','got','kind','know','day','think','continue','look','year','years','new','use','there','want','wanted','due','become','organ']
new_stopwords_list = stopwords.extend(new_stopwords)
def remove_stopwords(sentences):
    return [[word for word in simple_preprocess(str(doc)) if word not in new_stopwords_list] for doc in texts]



In [25]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['we', 'reported', 'last', 'quarter', 'that', 'we', 'have', 'penetration', 'in', 'terms', 'of', 'patients', 'of', 'about', 'we', 'have', 'seen', 'in', 'the', 'fourth', 'quarter', 'this', 'penetration', 'is', 'stable', 'and', 'is', 'something', 'that', 'in', 'some', 'way', 'was', 'expected', 'because', 'its', 'quarter', 'in', 'where', 'most', 'of', 'the', 'use', 'is', 'in', 'acute', 'while', 'im', 'sorry', 'chronic', 'while', 'we', 'have', 'seen', 'increase', 'in', 'the', 'penetration', 'because', 'of', 'the', 'use', 'of', 'acute', 'and', 'seasonal']]


In [26]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out


nlp = spacy.load('en', disable=['parser', 'ner'])



data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

['report last quarter have penetration term patient have see fourth quarter penetration be stable be something way be expect quarter where most use be acute be sorry chronic have see increase penetration use acute seasonal', 'when come expect end reality be share be really sort of quarter quarter basis last quarter quarter have be relatively stable do say be down little bit over']


### Create CountVectorizer

In [27]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=5,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             max_features=2000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)

### Build LDA model

In [28]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=5,               # Number of topics
                                      max_iter=15,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=20,            # n docs in each learning iter
                                      evaluate_every = 1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model) 

LatentDirichletAllocation(batch_size=20, doc_topic_prior=None,
             evaluate_every=1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=15, mean_change_tol=0.001,
             n_components=5, n_jobs=-1, n_topics=None, perp_tol=0.1,
             random_state=100, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)


In [9]:
print("Log Likelihood: ", lda_model.score(data_vectorized))

print("Perplexity: ", lda_model.perplexity(data_vectorized))


pprint(lda_model.get_params())

Log Likelihood:  -218047.46979508433
Perplexity:  620.7575179892256
{'batch_size': 20,
 'doc_topic_prior': None,
 'evaluate_every': 1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 15,
 'mean_change_tol': 0.001,
 'n_components': 5,
 'n_jobs': -1,
 'n_topics': None,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


### Use Grid Search to find the best Number of topic

In [29]:
search_params = {'n_components': [5, 10, 15, 20, 25], 'learning_decay': [.5, .7, .9]}


lda = LatentDirichletAllocation()


model = GridSearchCV(lda, param_grid = search_params)


model.fit(data_vectorized)

GridSearchCV(cv=None, error_score='raise',
       estimator=LatentDirichletAllocation(batch_size=20, doc_topic_prior=None,
             evaluate_every=1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=15,
             mean_change_tol=0.001, n_components=5, n_jobs=-1,
             n_topics=None, perp_tol=0.1, random_state=100,
             topic_word_prior=None, total_samples=1000.0, verbose=0),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_topics': [5, 10, 15, 20, 25], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)


GridSearchCV(cv=None, error_score='raise',
       estimator=LatentDirichletAllocation(batch_size=20, doc_topic_prior=None,
             evaluate_every=1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=15,
             mean_change_tol=0.001, n_components=5, n_jobs=-1,
             n_topics=None, perp_tol=0.1, random_state=100,
             topic_word_prior=None, total_samples=1000.0, verbose=0),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_topics': [5, 10, 15, 20, 25], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### Grid Search Result

In [30]:
best_lda_model = model.best_estimator_

print(best_lda_model)


print("Best Model's Params: ", model.best_params_)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.5,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=5, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)
Best Model's Params:  {'learning_decay': 0.5, 'n_components': 5}
Best Log Likelihood Score:  -78340.51801766852
Model Perplexity:  578.2324973579251


In [ ]:
n_components = [5, 10, 15, 20, 25]
print(best_lda_model.n_components)

5


In [ ]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

Best Model's Params:  {'learning_decay': 0.5, 'n_components': 5}
Best Log Likelihood Score:  -78190.31460441681
Model Perplexity:  578.5337569853338


### Get dominant topic for each document

In [ ]:
lda_output = best_lda_model.transform(data_vectorized)


topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]


docnames = ["Doc" + str(i) for i in range(len(data))]


df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic


def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)


df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
Doc0,0.01,0.01,0.01,0.01,0.95,4
Doc1,0.01,0.95,0.01,0.01,0.01,1
Doc2,0.01,0.49,0.01,0.01,0.48,1
Doc3,0.01,0.01,0.46,0.01,0.52,4
Doc4,0.01,0.01,0.16,0.8,0.01,3
Doc5,0.01,0.01,0.63,0.01,0.35,2
Doc6,0.01,0.01,0.96,0.01,0.01,2
Doc7,0.14,0.31,0,0,0.54,4
Doc8,0.3,0.48,0.01,0.01,0.21,1
Doc9,0.01,0.01,0.01,0.01,0.95,4


### Count number of document by topic number

In [ ]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,1,380
1,2,313
2,0,236
3,4,202
4,3,191


### pyLDAvis plot

In [ ]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel

/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
2      141.019379  -70.189392       1        1  25.918454
1       60.235168   35.194656       2        1  24.608941
0      -65.052025    9.075212       3        1  18.286464
4      -97.008659 -119.806450       4        1  15.989752
3       30.405968 -135.297348       5        1  15.196390, topic_info=     Category        Freq           Term       Total  loglift  logprob
term                                                                  
807   Default  204.000000           rate  204.000000  30.0000  30.0000
1014  Default  290.000000          think  290.000000  29.0000  29.0000
280   Default  104.000000       dividend  104.000000  28.0000  28.0000
799   Default  508.000000        quarter  508.000000  27.0000  27.0000
590   Default  273.000000         margin  273.000000  26.0000  26.0000
1098  Default  798.000000           year  798.000000  25.0000  25.0000
124   Default  148.000000           cash  148.000000  24.0000  24.0000
399   Default  115.000000           flow  115.000000  23.0000  23.0000
890   Default  166.000000            say  166.000000  22.0000  22.0000
815   Default  116.000000         really  116.000000  21.0000  21.0000
360   Default  264.000000         expect  264.000000  20.0000  20.0000
256   Default   64.000000        deposit   64.000000  19.0000  19.0000
496   Default  292.000000       increase  292.000000  18.0000  18.0000
915   Default  129.000000          share  129.000000  17.0000  17.0000
225   Default   81.000000         credit   81.000000  16.0000  16.0000
757   Default   92.000000         pretty   92.000000  15.0000  15.0000
116   Default  131.000000        capital  131.000000  14.0000  14.0000
847   Default  249.000000         remain  249.000000  13.0000  13.0000
675   Default   99.000000    opportunity   99.000000  12.0000  12.0000
568   Default  116.000000           loan  116.000000  11.0000  11.0000
1002  Default   84.000000            tax   84.000000  10.0000  10.0000
216   Default  162.000000           cost  162.000000   9.0000   9.0000
640   Default  100.000000            net  100.000000   8.0000   8.0000
531   Default  164.000000           just  164.000000   7.0000   7.0000
565   Default   70.000000         little   70.000000   6.0000   6.0000
732   Default  138.000000          point  138.000000   5.0000   5.0000
456   Default   57.000000           half   57.000000   4.0000   4.0000
83    Default  120.000000          basis  120.000000   3.0000   3.0000
874   Default  254.000000        revenue  254.000000   2.0000   2.0000
572   Default  221.000000           look  221.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
244    Topic5   18.976113       decrease   31.093167   1.3903  -5.6040
447    Topic5   24.494113          group   43.516074   1.3094  -5.3488
452    Topic5   39.873774       guidance   81.026924   1.1750  -4.8615
847    Topic5   91.369098         remain  249.945236   0.8778  -4.0323
428    Topic5   14.902439            gas   23.399099   1.4329  -5.8457
1098   Topic5  215.678098           year  798.288818   0.5754  -3.1734
360    Topic5   93.744456         expect  264.159933   0.8481  -4.0066
485    Topic5   47.539274         impact  111.156340   1.0347  -4.6856
57     Topic5   26.707986  approximately   51.889695   1.2200  -5.2622
689    Topic5   36.360471        outlook   82.733280   1.0620  -4.9537
799    Topic5  126.619313        quarter  508.643335   0.4936  -3.7060
887    Topic5   48.379733           sale  126.627603   0.9219  -4.6681
806    Topic5   31.578843          range   69.764778   1.0915  -5.0947
874    Topic5   74.102458        revenue  254.739081   0.6493  -4.2418
496    Topic5   78.645348       increase  292.185685   0.5717  -4.1823
760    Topic5   42.930698          price  118.482751   0.8689  -4.7876
955    Topic5  130.016368         stable  876.069594  -0.0237  -3.6795
4

### All Keywords 

In [ ]:
df_topic_keywords = pd.DataFrame(best_lda_model.components_)


df_topic_keywords.columns =  vectorizer.get_feature_names()
df_topic_keywords.index = topicnames
   

df_topic_keywords.head()

,ability,able,absolute,accelerate,acceleration,access,account,accounting,accretion,accretive,...,wide,win,winter,wonder,word,work,workforce,world,year,yield
Topic0,9.282649,18.263613,0.201340,0.203796,0.205673,0.200338,9.604146,3.797019,0.219429,0.200987,...,6.919834,0.201874,4.068437,0.200317,1.198911,4.979081,0.201093,2.741801,153.212315,27.194453
Topic1,0.201205,12.165214,7.189684,6.452924,2.918463,0.200853,6.133051,0.200957,0.207856,5.032343,...,1.020938,3.576727,1.198718,3.651278,1.535068,20.741365,0.200003,3.024412,210.797619,2.461255
Topic2,17.712293,22.153912,1.199715,12.844614,1.769446,5.027413,5.218685,0.200136,5.172211,0.210125,...,0.658606,22.819638,2.314867,0.200430,2.863762,33.219798,1.426577,6.859055,86.357759,3.034777
Topic3,0.201788,0.203006,0.202213,1.692715,1.906409,0.203937,2.842809,0.204979,0.200497,1.356540,...,0.200564,0.201082,0.217971,3.293401,0.201363,0.590648,1.199212,1.394530,220.645155,0.233023
Topic4,2.602066,0.214255,0.207047,1.805951,0.200010,2.367458,0.201310,2.596908,0.200007,0.200005,...,0.200058,0.200680,0.200008,2.654575,0.200895,6.469108,2.973115,1.980201,151.987152,2.076491


### 

In [ ]:
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=10)        

df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns =  ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords.head()



,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9
Topic 0,rate,stable,year,margin,quarter,expect,cost,growth,high,loan
Topic 1,stable,think,quarter,year,business,say,look,steady,market,growth
Topic 2,continue,growth,business,stable,steady,new,customer,grow,market,opportunity
Topic 3,year,stable,quarter,expect,remain,steady,increase,revenue,credit,growth
Topic 4,stable,year,dividend,cash,term,increase,share,flow,steady,result


In [12]:
nlp = spacy.load('en', disable=['parser', 'ner'])

def predict_topic(sentences, nlp=nlp):
    global sent_to_words
    global lemmatization

In [13]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['we', 'reported', 'last', 'quarter', 'that', 'we', 'have', 'penetration', 'in', 'terms', 'of', 'patients', 'of', 'about', 'we', 'have', 'seen', 'in', 'the', 'fourth', 'quarter', 'this', 'penetration', 'is', 'stable', 'and', 'is', 'something', 'that', 'in', 'some', 'way', 'was', 'expected', 'because', 'its', 'quarter', 'in', 'where', 'most', 'of', 'the', 'use', 'is', 'in', 'acute', 'while', 'im', 'sorry', 'chronic', 'while', 'we', 'have', 'seen', 'increase', 'in', 'the', 'penetration', 'because', 'of', 'the', 'use', 'of', 'acute', 'and', 'seasonal']]


In [14]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

nlp = spacy.load('en', disable=['parser', 'ner'])



data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])



In [15]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=5,                         
                             stop_words='english',             
                             lowercase=True,                   
                             token_pattern='[a-zA-Z0-9]{3,}',  
                             max_features=2000,            
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)
print(data_vectorized)

  (0, 894)	1
  (0, 496)	1
  (0, 1064)	2
  (0, 360)	1
  (0, 1084)	1
  (0, 955)	1
  (0, 413)	1
  (0, 705)	1
  (0, 1010)	1
  (0, 711)	3
  (0, 799)	3
  (0, 856)	1
  (1, 91)	1
  (1, 565)	1
  (1, 890)	1
  (1, 843)	1
  (1, 83)	1
  (1, 940)	1
  (1, 815)	1
  (1, 915)	1
  (1, 813)	1
  (1, 318)	1
  (1, 159)	1
  (1, 360)	1
  (1, 955)	1
  :	:
  (1320, 571)	1
  (1320, 494)	1
  (1320, 216)	3
  (1320, 965)	1
  (1320, 592)	1
  (1320, 80)	1
  (1320, 360)	1
  (1320, 1010)	2
  (1321, 369)	1
  (1321, 183)	1
  (1321, 78)	1
  (1321, 628)	1
  (1321, 960)	1
  (1321, 757)	1
  (1321, 262)	1
  (1321, 876)	1
  (1321, 954)	1
  (1321, 911)	1
  (1321, 1014)	1
  (1321, 106)	1
  (1321, 495)	2
  (1321, 1098)	1
  (1321, 955)	1
  (1321, 413)	1
  (1321, 799)	1


In [31]:
nlp = spacy.load('en', disable=['parser', 'ner'])

def predict_topic(sentences, lda):
    global sent_to_words
    global lemmatization

In [32]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['we', 'reported', 'last', 'quarter', 'that', 'we', 'have', 'penetration', 'in', 'terms', 'of', 'patients', 'of', 'about', 'we', 'have', 'seen', 'in', 'the', 'fourth', 'quarter', 'this', 'penetration', 'is', 'stable', 'and', 'is', 'something', 'that', 'in', 'some', 'way', 'was', 'expected', 'because', 'its', 'quarter', 'in', 'where', 'most', 'of', 'the', 'use', 'is', 'in', 'acute', 'while', 'im', 'sorry', 'chronic', 'while', 'we', 'have', 'seen', 'increase', 'in', 'the', 'penetration', 'because', 'of', 'the', 'use', 'of', 'acute', 'and', 'seasonal']]


In [33]:
def lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in data_words:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])



# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

['report last quarter have penetration term patient have see fourth quarter penetration be stable be something way be expect quarter where most use be acute be sorry chronic have see increase penetration use acute seasonal']


In [34]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=5,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             max_features=2000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [36]:
text = ', adding scientific talent and advance metrology capabilities. We believe these investments will allow us to respond faster to our customers and further differentiate us from our competitors. In terms of acquisitions, we completed two small transactions in the past 12 months.'
x = best_lda_model.transform(vectorizer.transform([text]))[0]
print(x)

[0.0099215  0.00977027 0.25415824 0.33509478 0.39105521]


In [ ]:
text=input('Enter a sentence : , adding scientific talent and advance metrology capabilities. We believe these investments will allow us to respond faster to our customers and further differentiate us from our competitors. In terms of acquisitions, we completed two small transactions in the past 12 months.)
x = best_lda_model.transform(vectorizer.transform([text]))[0]
#print(x)

count=0
for i in range(5):
    if x[i]>=0.6:
        count=count+1

if count>0:
    print ('Relevant to Management Opportunity')
else:
    
    print('Non Relevant to Management Opportunity')
    